# 06wk-022: 취업+각종영어점수, 다중공선성

최규빈  
2023-10-05

# 1. 강의영상

# 2. Imports

In [4]:
import numpy as np
import pandas as pd
import sklearn.linear_model

# 3. 상상

`-` 아래와 같은 가짜뉴스를 읽어보자.[1]

**헤드라인**: “텝스와 다른 영어 인증 시험들, 결국은 토익과 비슷한 결과를
보여준다?”

------------------------------------------------------------------------

**본문**:

최근 몇 년 동안, 토익의 신뢰성에 대한 논란이 계속되어 왔습니다. 이러한
배경 속에서 텝스(TEPS), 토플(TOEFL) 등 여러 새로운 영어 능력 평가 시험이
등장하였습니다. 많은 학생들과 직장인들은 이러한 새로운 시험들이 토익보다
더 신뢰성 있고 현실적인 능력을 평가할 것이라는 기대감을 가지고
있었습니다.

그러나 최근에 발표된 연구결과에 따르면, 텝스와 다른 영어 인증 시험들도
결국에는 토익과 매우 비슷한 성적 분포와 결과를 보여주었다고 합니다. 연구
팀은 여러 시험들간의 점수 분포와 성적의 상관관계를 분석한 결과, 대부분의
시험들이 실제 영어 능력에 대해 유사한 평가를 제공한다는 결론을
내렸습니다.

“많은 사람들이 새로운 시험들이 더 현실적이거나 다양한 영어 능력을 평가할
것이라 기대했지만, 실제로는 모든 시험들이 비슷한 결과를 보여주었습니다.”
라며 연구 팀의 대표는 이렇게 언급하였습니다.

이러한 연구결과는 영어 능력 평가 시험의 표준화와 신뢰성에 대한 논의를
새롭게 불러일으킬 것으로 보입니다.

`-` 이 뉴스에 근거하여 아래의 가짜자료를 생성했다.

[1] 제가 ChatGPT를 이용하여 생성한 가짜뉴스입니다. 뉴스는 실제사실을
반영한 것이 아닙니다.

In [112]:
df = pd.read_csv("https://raw.githubusercontent.com/guebin/MP2023/main/posts/employment_manytoeic.csv")
df

-   toeic0,toeic1,… 등은 토익과 유사한 새로운 시험들을 의미함. (실제
    점수는 정수이겠지만 편의상 소수점으로 사용하였음)
-   employmet_score는 회사에 내부적으로 정의된 채점기준표이며, 이
    기준표에 의하여 일정점수 이상인 사람은 합격하고, 그렇지 않은 사람은
    불합격한다고 가정함.

`-` 비밀: 사실 저는 데이터를 아래의 공식에 따라서 만들었어요..

    employment_score = toeic*1/200 + gpa*1.3 + random 

# 4. 잘못된 분석

In [144]:
df_train, df_test = sklearn.model_selection.train_test_split(df,test_size=0.3)

In [145]:
X = df_train.drop(['employment_score'],axis=1)
y = df_train[['employment_score']]
XX = df_test.drop(['employment_score'],axis=1)
yy = df_test[['employment_score']]

In [146]:
predictr = sklearn.linear_model.LinearRegression()

In [147]:
X.shape

In [148]:
predictr.fit(X,y)

In [149]:
predictr.coef_,predictr.intercept_

-   실제계수값은 `토익*0.005`, `GPA*1.3` 인데.. GPA는 얼추맞는거 같지만,
    토익은 딱봐도 틀려보임..

In [150]:
predictr.score(X,y) 

In [151]:
predictr.score(XX,yy)

# 5. 제대로 분석했다면?

## A. `toeic`과 `gpa`가 유의미한 변수라는걸 눈치챘다면?

In [122]:
X = df_train[['toeic','gpa']]
y = df_train[['employment_score']]
XX = df_test[['toeic','gpa']]
yy = df_test[['employment_score']]

In [123]:
predictr = sklearn.linear_model.LinearRegression()

In [124]:
predictr.fit(X,y)

In [125]:
predictr.coef_,predictr.intercept_

In [126]:
predictr.score(X,y)

In [127]:
predictr.score(XX,yy)

## B. 하다못해 `toeic0`과 `gpa`로 적합했다면?

In [128]:
X = df_train[['toeic0','gpa']]
y = df_train[['employment_score']]
XX = df_test[['toeic0','gpa']]
yy = df_test[['employment_score']]

In [129]:
predictr = sklearn.linear_model.LinearRegression()

In [130]:
predictr.fit(X,y)

In [131]:
predictr.coef_,predictr.intercept_

In [132]:
predictr.score(X,y)

In [133]:
predictr.score(XX,yy)

# 6. Discussions

`-` 어렴풋한 감각: 쓸모없는 변수가 많이 있으면 오히려 모형의 적합도가
떨어진다. (규칙을 찾으면 안될것에서 규칙을 찾고있으니까 잘 될리가 없지)

`-` 쓸모없는 변수란 느낌이 드는 경우?

-   경우1: 진짜 쓸모 없는거.. (X1= 부먹/찍먹, X2= 민초/민초X) $\to$
    애초에 `X1`,`X2`을 보고 `y`를 맞출 생각이 들지 않어..
-   경우2: 실제론 쓸모 있는데, 대체자가 있는 경우. (X1= toeic, X2=
    유사toeic) $\to$ `X1` 을 보고 `y`를 맞출 것 같은 생각이 들어, 그리고
    `X2`를 보고 `y`를 맞출 것 같은 생각도 들어. 그런데 `X1`이랑 `X2`는
    너무 비슷해

`-` 경우1, 경우2 모두 과대적합(overfitting)을 야기한다. 그리고 경우2와
같은 상황에서 발생하는 문제를 특별히 다중공선성이라고 칭한다.

`-` 다중공선성이 발생했을 경우 나타나는 특징

1.  계수들의 추정값이 불안정하다[1]
2.  과대적합 문제를 불러올 수 있음
3.  계수의 해석이 어려움

`-` 의문: 그래서 해결방법은? 변수를 잘 보고 제거하라고??

[1] 사실 `toeic의 계수 + toeic0의 계수 + ... + toeic49의 계수` $\approx$
`0.05` 이면 거진 올바른 추정아니야? 애초에 답이 여러개야